# Sistema Masa-Resorte en 2D

## Análisis Teórico

El sistema de una masa $m$ unida a un resorte sin masa en un espacio bidimensional se modela con la **segunda ley de Newton** y la **ley de Hooke**. Definamos las ecuaciones del movimiento.

---

### **1. Formulación del problema**
La masa está sometida a dos fuerzas principales:

- **Fuerza del resorte**: Según la ley de Hooke:
  $$
  \mathbf{F}_s = -k (\ell - \ell_0) \hat{r}
  $$
  donde $k$ es la constante del resorte, $\ell$ es la distancia de la masa al origen y $\ell_0$ es la longitud en reposo del resorte. La dirección del resorte está dada por el vector unitario $\hat{r}$.

- **Fuerza de amortiguamiento** (si se considera un medio viscoso con coeficiente de amortiguamiento $b$):
  $$
  \mathbf{F}_d = -b \mathbf{v}
  $$

Aplicando la segunda ley de Newton:
  $$
  m \frac{d^2 \mathbf{r}}{dt^2} = -k (\ell - \ell_0) \hat{r} - b \frac{d\mathbf{r}}{dt}
  $$

Dado que el movimiento es en **2D**, expresamos la posición en coordenadas polares:
$$
\mathbf{r} = r \hat{r}, \quad \mathbf{v} = \dot{r} \hat{r} + r \dot{\theta} \hat{\theta}
$$

La aceleración en coordenadas polares es:
$$
\mathbf{a} = (\ddot{r} - r\dot{\theta}^2) \hat{r} + (r\ddot{\theta} + 2\dot{r} \dot{\theta}) \hat{\theta}
$$

---

### **2. Ecuaciones diferenciales del movimiento**
Descomponiendo en componentes radiales y angulares:

1. **Ecuación radial**:
   $$
   m(\ddot{r} - r\dot{\theta}^2) = -k(r - r_0) - b\dot{r}
   $$
   O, reescribiendo:
   $$
   \ddot{r} + \frac{b}{m} \dot{r} + \frac{k}{m} (r - r_0) = r\dot{\theta}^2
   $$

2. **Ecuación angular** (por conservación del momento angular si no hay fuerzas externas en $\hat{\theta}$):
   $$
   m(r\ddot{\theta} + 2\dot{r} \dot{\theta}) = 0
   $$
   $$
   r\ddot{\theta} + 2\dot{r} \dot{\theta} = 0
   $$

---

### **3. Solución Analítica**
Sin amortiguamiento ($b = 0$) y asumiendo que el término $r\dot{\theta}^2$ es pequeño, la ecuación radial se reduce a una ecuación diferencial de oscilador armónico:
$$
\ddot{r} + \frac{k}{m} (r - r_0) = 0
$$

La solución general es:
$$
r(t) = r_0 + A \cos(\omega t + \phi)
$$
donde $A$ y $\phi$ dependen de las condiciones iniciales, y la frecuencia natural del sistema es:
$$
\omega = \sqrt{\frac{k}{m}}
$$

Para la ecuación angular:
$$
\frac{d}{dt} (r^2 \dot{\theta}) = 0 \quad \Rightarrow \quad r^2 \dot{\theta} = L
$$
donde $L$ es el momento angular por unidad de masa. Esto sugiere que el movimiento sigue una trayectoria elíptica o circular dependiendo de las condiciones iniciales.

Vemos que: 
- La ecuación radial es una oscilación armónica simple con desplazamiento alrededor de $r_0$.
- La ecuación angular indica conservación del momento angular, lo que implica que la masa sigue un movimiento oscilatorio en torno a $r_0$, describiendo una trayectoria cerrada si $L \neq 0$.

El sistema tiene **solución analítica exacta** en términos de funciones trigonométricas, siempre que la fuerza elástica sea la única fuerza presente o el amortiguamiento sea lineal.


In [20]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mass_spring_system import Mass, Spring, System

In [42]:
# Plotting functions

def plot_trajectory(x, y, save=False, show=True, imname=""):
    """Grafica la trayectoria de un sistema masa-resorte 2D."""
    plt.figure(figsize=(8, 6))
    plt.plot(x[0], y[0], "ro", label="Initial point")  
    plt.plot(x[-1], y[-1], "gx", label="Final point")
    plt.plot(x, y, label="Trajectory")
    plt.title("2D Mass-Spring System")
    plt.xlabel("Position X (m)")
    plt.ylabel("Position Y (m)")
    plt.grid()
    plt.legend()

    if save:
        plt.savefig(f"plots/trajectory{imname}.png", dpi=300)
    if show:
        plt.show()
    plt.close()

def plot_3d(t, x, y, save=False, show=True, imname=""):
    """Grafica la trayectoria en 3D con el tiempo en el eje Z."""
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x[0], y[0], t[0], color="red", label="Initial point")
    ax.scatter(x[-1], y[-1], t[-1], color="blue", label="Final point")
    ax.plot(x, y, t, label="Trajectory")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Time")
    plt.title("3D Trajectory")

    if save:
        plt.savefig(f"plots/trajectory_3d{imname}.png", dpi=300)
    if show:
        plt.show()
    plt.close()

def plot_phase_space(x, vx, y, vy, save=False, show=True, imname=""):
    """Grafica el espacio fase (posición vs velocidad) en 2D."""
    plt.figure(figsize=(12, 6))

    # x vs vx
    plt.subplot(1, 2, 1)
    plt.plot(x, vx, label="x vs vx", color="red")
    plt.xlabel("Position (m)")
    plt.ylabel("Velocity (m/s)")
    plt.title("Phase Space: x vs vx")
    plt.grid()
    plt.legend()
    
    # y vs vy
    plt.subplot(1, 2, 2)
    plt.plot(y, vy, label="y vs vy", color="blue")
    plt.xlabel("Position (m)")
    plt.ylabel("Velocity (m/s)")
    plt.title("Phase Space: y vs vy")
    plt.grid()
    plt.legend()

    if save:
        plt.savefig(f"plots/phase_space{imname}.png", dpi=300)
    if show:
        plt.show()
    plt.close()

def plot_energy_time(t, kin, pot, tot, save=False, show=True, imname=""):
    """Grafica la evolución de la energía en el tiempo."""
    plt.figure(figsize=(8, 6))
    plt.plot(t, kin, label="Kinetic Energy", color="orange")
    plt.plot(t, pot, label="Potential Energy", color="purple")
    plt.plot(t, tot, label="Total Energy", color="black", linestyle="dashed")
    plt.xlabel("Time (s)")
    plt.ylabel("Energy (J)")
    plt.title("System Energy")
    plt.grid()
    plt.legend()

    if save:
        plt.savefig(f"plots/energy_time{imname}.png", dpi=300)
    if show:
        plt.show()
    plt.close()


In [43]:
def cartesian_to_polar(x, y):
    r = np.sqrt(x**2 + y**2)  # Radio
    theta = np.arctan2(y, x)  # Ángulo (en radianes)
    return r, theta

In [44]:
def load_data(file_path="data/data.dat"):
    """Carga los datos desde el archivo .dat"""
    data = np.loadtxt(file_path, skiprows=1)
    t, x, y, vx, vy, E_kin, E_pot, E_total = data.T
    return t, x, y, vx, vy, E_kin, E_pot, E_total

In [45]:
# System parameters: 
mass = Mass(position=[1.0, 0.0], velocity=[1.0, 2.0], mass=1.0)
spring = Spring(k=10.0, rest_length=1.0)

# Simulation parameters
t_max = 20.0
dt = 0.01
t_eval = np.arange(0, t_max, dt)

system_damped = System(mass=mass, spring=spring, damping=0.1)
system_regular = System(mass=mass, spring=spring, damping=0.0)

### Simulación teórica: 

In [36]:
from scipy.integrate import solve_ivp

In [46]:
# Ecuaciones de movimiento
def equations_of_motion(t, y, system):
    x, y_pos, vx, vy = y
    r = np.sqrt(x**2 + y_pos**2)
    
    # Hooke's law
    force_magnitude = -system.spring.k * (r - system.spring.rest_length) / r
    fx = force_magnitude * x
    fy = force_magnitude * y_pos
    
    # Damping force
    fx -= system.damping * vx
    fy -= system.damping * vy
    
    ax = fx / system.mass.mass
    ay = fy / system.mass.mass
    
    return [vx, vy, ax, ay]


In [47]:
# Solve the system without damping
sol_regular = solve_ivp(equations_of_motion, [0, t_max],
                        [mass.position[0], mass.position[1], mass.velocity[0], mass.velocity[1]],
                        args=(system_regular,), t_eval=t_eval, method='RK45')

# Solve the system with damping
sol_damped = solve_ivp(equations_of_motion, [0, t_max],
                        [mass.position[0], mass.position[1], mass.velocity[0], mass.velocity[1]],
                        args=(system_damped,), t_eval=t_eval, method='RK45')

In [48]:
omega = np.sqrt(spring.k / mass.mass)
analytical_r = spring.rest_length + np.cos(omega * t_eval)

### Simulación numérica: 

In [ ]:
system_damped.simulate(t_max, dt, datapath="damped_data.dat")
system_regular.simulate(t_max, dt, datapath="regular_data.dat")

In [52]:
# Generate plots: Damped data
t, x, y, vx, vy, E_kin, E_pot, E_total = load_data(file_path="data/damped_data.dat")
imname = "_damped"
plot_trajectory(x, y, save=True, show=False, imname=imname)
plot_phase_space(x, vx, y, vy, save=True, show=False, imname=imname)
plot_energy_time(t, E_kin, E_pot, E_total, save=True, show=False, imname=imname)

In [53]:
# Generate plots: Damped data
t, x, y, vx, vy, E_kin, E_pot, E_total = load_data(file_path="data/regular_data.dat")
imname = "_regular"
plot_trajectory(x, y, save=True, show=False, imname=imname)
plot_phase_space(x, vx, y, vy, save=True, show=False, imname=imname)
plot_energy_time(t, E_kin, E_pot, E_total, save=True, show=False, imname=imname)

## Análisis de Resultados: Oscilación Simple vs Oscilación Forzada

### Trayectorias en el espacio de configuración

<div style="display: flex; justify-content: center; text-align: center;">
    <div style="margin: 10px;">
        <img src="plots/trajectory_regular.png" width="100%">
    </div>
    <div style="margin: 10px;">
        <img src="plots/trajectory_damped.png" width="100%">
    </div>
</div>

### Espacio fase

<div style="display: flex; justify-content: center; text-align: center;">
    <div style="margin: 10px;">
        <img src="plots/phase_space_regular.png" width="100%">
    </div>
    <div style="margin: 10px;">
        <img src="plots/phase_space_damped.png" width="100%">
    </div>
</div>

### Energía

<div style="display: flex; justify-content: center; text-align: center;">
    <div style="margin: 10px;">
        <img src="plots/energy_time_regular.png" width="100%">
    </div>
    <div style="margin: 10px;">
        <img src="plots/energy_time_damped.png" width="100%">
    </div>
</div>